In [1]:
import os

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [5]:
model = ChatGoogleGenerativeAI(model='gemini-pro', convert_system_message_to_human= True)

In [6]:
response = model.invoke("Hello, world!").content

In [7]:
from langchain_core.output_parsers import StrOutputParser

In [8]:
parser = StrOutputParser()

In [9]:
chain = model | parser 

In [10]:
chain.invoke("Hello, world!")

'Greetings, Earth inhabitants!'

In [11]:
from langchain_core.messages import HumanMessage

In [12]:
while True:
    message = input("Write your query")
    if message == "Bye":
        break
    else:
        print(chain.invoke([HumanMessage(content=message)]))

In [13]:
from langchain_core.chat_history import BaseChatMessageHistory

In [14]:
from langchain_core.chat_history import InMemoryChatMessageHistory

In [15]:
from langchain_core.runnables.history import RunnableWithMessageHistory

In [16]:
from langchain_core.messages import AIMessage

In [17]:
output = model.invoke([
        HumanMessage(content="Hi! I'm cristinao"),
        AIMessage(content="Hello cristinao! How can I assist you today?"),
        HumanMessage(content="What's my name?") 
])

In [18]:
parser.invoke(output)

'You did not provide me with your name, so I cannot answer that question.'

In [19]:
store ={}

In [20]:
def get_session_history(session_id: str) ->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [21]:
model_with_memory = RunnableWithMessageHistory(model, get_session_history)

In [22]:
config = {"configurable": {"session_id": "firstchat"}}

In [23]:
model_with_memory.invoke([HumanMessage(content="Hi! I am Samin Sadaf")], config=config).content

"Hello, Samin. It's nice to meet you!"

In [24]:
model_with_memory.invoke([HumanMessage(content="What is my name?")], config=config).content

'Your name is Samin Sadaf.'

In [25]:
config = {"configurable": {"session_id": "secondchat"}}

In [26]:
model_with_memory.invoke([HumanMessage(content="What is my name?")], config=config).content

'I do not have access to your personal information, therefore I cannot provide your name.'

In [27]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi! I am Samin Sadaf'), AIMessage(content="Hello, Samin. It's nice to meet you!", response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-f58a5cec-6e9b-478c-bda4-473126e6c7bc-0', usage_metadata={'input_tokens': 9, 'output_tokens': 13, 'total_tokens': 22}), HumanMessage(content='What is my name?'), AIMessage(content='Your name is Samin Sadaf.', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'cate

In [28]:
config = {"configurable": {"session_id": "firstchat"}}

In [29]:
model_with_memory.invoke([HumanMessage(content="What did I ask you last time?What was your answer?")], config=config).content

'Last time, you asked me "What is my name?". I answered "Your name is Samin Sadaf.".'

In [30]:
config = {"configurable": {"session_id": "secondchat"}}

In [31]:
model_with_memory.invoke([HumanMessage(content="What did I ask you last time?What was your answer?")], config=config).content

'I do not have access to your previous interactions or conversations, so I cannot provide information about what you asked me last time or what my answer was.'

In [32]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi! I am Samin Sadaf'), AIMessage(content="Hello, Samin. It's nice to meet you!", response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-f58a5cec-6e9b-478c-bda4-473126e6c7bc-0', usage_metadata={'input_tokens': 9, 'output_tokens': 13, 'total_tokens': 22}), HumanMessage(content='What is my name?'), AIMessage(content='Your name is Samin Sadaf.', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'cate

In [33]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [34]:
prompt=ChatPromptTemplate.from_messages(
    [
      ("system","You are a helpful assistant. Answer all questions to the best of your ability.",),
      MessagesPlaceholder(variable_name="messages"),
    ]
)

In [35]:
chain = prompt | model | parser

In [36]:
chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]})

'Hi Bob! How can I help you today?'

In [37]:
chain.invoke({"messages": [HumanMessage(content="What is my name?")]})

'I do not have access to your personal information and cannot answer that question.'

In [38]:
chain_with_memory = RunnableWithMessageHistory(chain, get_session_history)

In [39]:
config = {"configurable":{"session_id":"thirdchat"}}

In [42]:
chain_with_memory.invoke({"messages": [HumanMessage(content="Hi! I'm bob")]}, config=config)

'Hello Bob. How can I help you today?'

In [43]:
chain_with_memory.invoke({"messages": [HumanMessage(content="What is my name?")]}, config=config)

'Your name is Bob.'

In [44]:
chain_with_memory.invoke({"messages": [HumanMessage(content="What is 2+2=?")]}, config=config)

'2 + 2 = 4'

In [46]:
chain_with_memory.invoke(
    [HumanMessage(content="what should i add in previous addition so that i will become 10?"),
     ],config=config
)

'You need to add 6 to the previous addition of 2 + 2 to get 10.\n\n2 + 2 = 4\n4 + 6 = 10'

In [47]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi! I am Samin Sadaf'), AIMessage(content="Hello, Samin. It's nice to meet you!", response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-f58a5cec-6e9b-478c-bda4-473126e6c7bc-0', usage_metadata={'input_tokens': 9, 'output_tokens': 13, 'total_tokens': 22}), HumanMessage(content='What is my name?'), AIMessage(content='Your name is Samin Sadaf.', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'cate

Trim conversation to be within the context window of the model

In [48]:
from langchain_core.messages import SystemMessage, trim_messages

In [49]:
trimmer = trim_messages(
    max_tokens = 40,
    strategy = 'last',
    token_counter = model,
    include_system = True,
    allow_partial = False,
    start_on = 'human',
)

In [50]:
messages = [
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [51]:
model.get_num_tokens_from_messages(messages)

51